In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path


In [ ]:
# Set up paths dynamically
current_dir = Path.cwd()
project_root = current_dir.parent if current_dir.name == 'code' else current_dir
code_dir = project_root / 'code'
data_dir = project_root / 'data'
figures_dir = project_root / 'figures'

# Create directories if they don't exist
data_dir.mkdir(exist_ok=True)
figures_dir.mkdir(exist_ok=True)

# Add code directory to Python path
sys.path.append(str(code_dir))

print(f"Project root: {project_root}")
print(f"Code directory: {code_dir}")
print(f"Data directory: {data_dir}")
print(f"Figures directory: {figures_dir}")


In [ ]:
# Import custom modules
from A_afp import AdversarialFairPayoffs
from B_bp import BernoulliPayoffs
# from C_pp import PachinkoPayoffs
# from D_ad import AdversarialGenerativeModel
from EW import ExponentialWeightsAlgorithm
from MC import monte_carlo_simulation

print("All modules imported successfully!")


In [ ]:
# Fixed parameters
k = 3  
n = 1000  
num_simulations = 100  # Monte Carlo simulation times

# Epsilon values
epsilon_values = {
    'random': 0.01,  
    'optimal': np.sqrt(np.log(k) / n), 
    'FTL': 100  
}

print(f"Epsilon values:")
for name, value in epsilon_values.items():
    print(f"  {name}: {value:.6f}")


In [ ]:
# AdversarialFairPayoffs environment

# Save results
results_data = []

for epsilon_name, epsilon_value in epsilon_values.items():
    print(f"\nRunning simulations for {epsilon_name} (epsilon = {epsilon_value:.6f})...")
    
    regret_histories = []
    total_payoffs = []
    
    for sim in range(num_simulations):
        env = AdversarialFairPayoffs(k)
        algorithm = ExponentialWeightsAlgorithm(k, epsilon=epsilon_value, n=n)
        
        # Run algorithm
        regret_history, total_payoff, action_history, cumulative_payoffs = algorithm.run_algorithm(env.generate_payoffs)
        
        regret_histories.append(regret_history)
        total_payoffs.append(total_payoff)
        
        if (sim + 1) % 20 == 0:
            print(f"  Completed {sim + 1}/{num_simulations} simulations")
    
    # Add results to dataframe
    regret_array = np.array(regret_histories)
    mean_regret = np.mean(regret_array, axis=0)
    std_regret = np.std(regret_array, axis=0)
    final_regrets = regret_array[:, -1]
    
    for round_num in range(n):
        results_data.append({
            'round': round_num + 1,
            'epsilon_type': epsilon_name,
            'epsilon_value': epsilon_value,
            'mean_regret': mean_regret[round_num],
            'std_regret': std_regret[round_num],
            'final_regret_mean': np.mean(final_regrets),
            'final_regret_std': np.std(final_regrets)
        })

print("\nAll simulations completed!")

results_df = pd.DataFrame(results_data)

# Save results to csv file
adversarial_csv_path = data_dir / 'adversarial_fair_payoffs_results.csv'
results_df.to_csv(adversarial_csv_path, index=False)
print(f"Results saved to {adversarial_csv_path}")

# Statistical results
print("\nFinal regret statistics:")
for epsilon_name in epsilon_values.keys():
    subset = results_df[results_df['epsilon_type'] == epsilon_name]
    final_stats = subset.iloc[0]  
    print(f"  {epsilon_name}: Mean = {final_stats['final_regret_mean']:.4f}, Std = {final_stats['final_regret_std']:.4f}")


In [ ]:
# BernoulliPayoffs environment

# Save results
results_data_bernoulli = []

for epsilon_name, epsilon_value in epsilon_values.items():
    print(f"\nRunning simulations for {epsilon_name} (epsilon = {epsilon_value:.6f})...")
    
    regret_histories = []
    total_payoffs = []
    
    for sim in range(num_simulations):
        env = BernoulliPayoffs(k)
        algorithm = ExponentialWeightsAlgorithm(k, epsilon=epsilon_value, n=n)
        
        # Run algorithm
        regret_history, total_payoff, action_history, cumulative_payoffs = algorithm.run_algorithm(env.generate_payoffs)
        
        regret_histories.append(regret_history)
        total_payoffs.append(total_payoff)
        
        if (sim + 1) % 20 == 0:
            print(f"  Completed {sim + 1}/{num_simulations} simulations")
    
    # Add results to dataframe
    regret_array = np.array(regret_histories)
    mean_regret = np.mean(regret_array, axis=0)
    std_regret = np.std(regret_array, axis=0)
    final_regrets = regret_array[:, -1]
    
    for round_num in range(n):
        results_data_bernoulli.append({
            'round': round_num + 1,
            'epsilon_type': epsilon_name,
            'epsilon_value': epsilon_value,
            'mean_regret': mean_regret[round_num],
            'std_regret': std_regret[round_num],
            'final_regret_mean': np.mean(final_regrets),
            'final_regret_std': np.std(final_regrets)
        })

print("\nAll simulations completed!")

results_df_bernoulli = pd.DataFrame(results_data_bernoulli)

# Save results to csv file
bernoulli_csv_path = data_dir / 'bernoulli_payoffs_results.csv'
results_df_bernoulli.to_csv(bernoulli_csv_path, index=False)
print(f"Results saved to {bernoulli_csv_path}")

# Statistical results
print("\nFinal regret statistics:")
for epsilon_name in epsilon_values.keys():
    subset = results_df_bernoulli[results_df_bernoulli['epsilon_type'] == epsilon_name]
    final_stats = subset.iloc[0]  
    print(f"  {epsilon_name}: Mean = {final_stats['final_regret_mean']:.4f}, Std = {final_stats['final_regret_std']:.4f}")


In [ ]:
# Import visualization functions and run plots
import visualization

# Load the data
adversarial_df = pd.read_csv(adversarial_csv_path)
bernoulli_df = pd.read_csv(bernoulli_csv_path)

# Use the plot_regret_comparison function from visualization.py
visualization.plot_regret_comparison(adversarial_df, 'AdversarialFairPayoffs Environment', 'adversarial_regret_comparison.png')
visualization.plot_regret_comparison(bernoulli_df, 'BernoulliPayoffs Environment', 'bernoulli_regret_comparison.png')
